# Voronoi diagram 0.0

In [1444]:
import math
import numpy as np

In [1445]:
class Point(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def str_point(self):
        return "["+str(round(self.x,3))+","+str(round(self.y,3))+"]"
        
    def is_x_greater(self, p):
        return self.x > p.x
    
    def is_y_greater(self, p):
        return self.y > p.y
        
    def delta_x(self, p):
        return self.x-p.x
    
    def delta_y(self, p):
        return self.y-p.y
    
    def dist_to_point(self, p):
        return math.sqrt(math.pow(self.x - p.x, 2) + math.pow(self.y - p.y, 2))
    
    def dist_to_par(self, focus): #se la distanza punto parabola è infinita (direttrice parabola passante per punto e fuoco)
        return math.pow(self.dist_to_point(focus), 2)/(2*abs(self.delta_y(focus))) if self.delta_y(focus)!=0 else math.inf
    
    def is_same_point(self, p):
        return self.x == p.x and self.y == p.y
        
    def is_in(self, ps):
        for p in ps:
            if self.is_same_point(p):
                return True
        return False

In [1446]:
def circumcenter(a, b, c):
    if isinstance(a, Segment):
        print("a is", a.str_segment())
    if isinstance(b, Segment):
        print("b is", b.str_segment())
    if isinstance(c, Segment):
        print("c is", c.str_segment())
    d = (a.x - c.x) * (b.y - c.y) - (b.x - c.x) * (a.y - c.y)
    if d==0: #tre punti allineati
        return Point(math.inf, math.inf)
    xc = (((a.x - c.x) * (a.x + c.x) + (a.y - c.y) * (a.y + c.y)) / 2 * 
          (b.y - c.y) - ((b.x - c.x) * (b.x + c.x) + (b.y - c.y) * (b.y + c.y)) / 2 * 
          (a.y - c.y))/d
    yc = (((b.x - c.x) * (b.x + c.x) + (b.y - c.y) * (b.y + c.y)) / 2 * 
          (a.x - c.x) - ((a.x - c.x) * (a.x + c.x) + (a.y - c.y) * (a.y + c.y)) / 2 * 
          (b.x - c.x))/d
    return Point(xc, yc)

def circumbottom(a, b, c):
    cc = circumcenter(a, b, c)
    return Point(cc.x, cc.y-cc.dist_to_point(a)) #(cc.x, cc.y-r)

def circumtop(a, b, c):
    cc = circumcenter(a, b, c)
    return Point(cc.x, cc.y+cc.dist_to_point(a)) #(cc.x, cc.y-r)

def arc_cross_x(f1, f2, q):
    if f1.y!=f2.y:
        s1=(f1.y*f2.x-f1.x*f2.y+f1.x*q-f2.x*q+math.sqrt((f1.x*f1.x+f1.y*f1.y-2*f1.x*f2.x+f2.x*f2.x-2*f1.y*f2.y+f2.y*f2.y)*(f1.y*f2.y-f1.y*q-f2.y*q+q*q)))/(f1.y-f2.y)  
        s2=(f1.y*f2.x-f1.x*f2.y+f1.x*q-f2.x*q-math.sqrt((f1.x*f1.x+f1.y*f1.y-2*f1.x*f2.x+f2.x*f2.x-2*f1.y*f2.y+f2.y*f2.y)*(f1.y*f2.y-f1.y*q-f2.y*q+q*q)))/(f1.y-f2.y)  
        return [s1, s2] if s1<s2 else [s2, s1]
    else:
        return [(f1.x+f2.x)/2, (f1.x+f2.x)/2]

def parabola_y(f, q, x):
    if q!=f.y:
        return math.pow((x - f.x),2)/(2*(abs(q-f.y)))+(q-f.y)/2
    else:
        return math.inf

def distance(a, b):
    return math.sqrt(math.pow((a.x - b.x), 2) + math.pow((a.y - b.y), 2))

def go_to_i(alist, test):
    i = 0
    while i < len(alist) and test(alist[i]):
        i += 1
    return i


In [1447]:
print("1st intersection:", arc_cross_x(Point(0,0), Point(4,2), 4)[0],". Second intersection: ",arc_cross_x(Point(0,0), Point(4,2), 4)[1])

1st intersection: 1.675444679663241 . Second intersection:  14.32455532033676


In [1448]:
f=Point(0,0)
q=2
x=2
print("Parabola with y =", q, "and focus in",f.str_point(),"evaluated in x =",x,"gives f(x) =", parabola_y(f,q,x))

Parabola with y = 2 and focus in [0,0] evaluated in x = 2 gives f(x) = 2.0


In [1449]:
class Segment(object):
    
    def __init__(self,  pl=None, pr=None, start=None, end=None):
        self.start = start #punto iniziale del segmento
        self.end = end #punto finale del segmento
        self.pl = pl #uno dei due punti di cui l'oggetto è un segmento dell'asse
        self.pr = pr #uno dei due punti di cui l'oggetto è un segmento dell'asse
        
    def str_segment(self):
        start="Start: None" if self.start==None else "Start: "+self.start.str_point()
        end="Theta: "+str(self.theta()) if self.end==None else "End: "+self.end.str_point()
        return start+"; "+end+"; left arc focus: "+self.pl.str_point()+"; right arc focus: "+self.pr.str_point()
    
    def halfplane(self):
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: i semisegmenti vengono specchiati rispetto a x se cambia il verso di sweep
        #per sweep verso x positive. +: semipiano superiore e x+; -: semipiano inferiore e x-
        #per sweep verso x negative. +: semipiano inferiore e x+; -: semipiano superiore e x- (possibile invertire la condizione)
        #L'assegnazione di x+ e x- è stata scelta arbitrariamente
        if self.pl.x<self.pr.x: #semipiano superiore
            return 2
        if self.pl.x>self.pr.x: #semipiano inferiore
            return -2
        if self.pl.x==self.pr.x and self.pl.y>self.pr.y: #asse x+
            return 1
        return -1 #asse x-
    
    def m(self): #y=mx+q ; m=-(pl.x-pr.x)/(pl.y-pr.y) ; q=(pl.y+pr.y)/2-m*(pl.x+pr.x)/2
        return math.inf if self.pl.y==self.pr.y else -(self.pl.x-self.pr.x)/(self.pl.y-self.pr.y)
    
    def direction(self):
        (m, hp)=(self.m(), self.halfplane())
        return Point(1,m) if (hp*m>0 or (m==0 and hp>0)) else Point(-1,-m) 
    
    def theta(self, rad=False):
        theta = round(math.atan2(self.direction().y, self.direction().x), 3)
        return round(theta*180/math.pi, 2) if rad==False else round(theta, 3)
        
    def y(self, x):
        return self.m()*x+self.start.y-self.m()*self.start.x
    
    def get_intersection(self, s1, s2=None):
        s2=self if s2==None else s2
        (s1_m, s2_m)=(s1.m(), s2.m())
        (s1_hp, s2_hp)=(s1.halfplane(), s2.halfplane())
        #print(str(s1_m), str(s2_m), str(s1_hp), str(s2_hp))
        if (s1_hp*s1_m>0 or (s1_m==0 and s1_hp>0)): #semiretta semipiano dx 
            s1_vec=Point(1,s1_m) 
        else: #semiretta nel semipiano sx
            s1_vec=Point(-1,-s1_m)
        if (s2_hp*s2_m>0 or (s2_m==0 and s2_hp>0)): #semiretta semipiano dx 
            s2_vec=Point(1,s2_m) 
        else: #semiretta nel semipiano sx
            s2_vec=Point(-1,-s2_m)
        dx = s2.start.x - s1.start.x
        dy = s2.start.y - s1.start.y
        if s2_m==math.inf: #segmento 2 verticale
            #print("Vertical:", s2_hp*(s1_m*dx-dy), s1_vec.x*dx)
            if s2_hp*(s1_m*dx-dy)>=-1.0e-10 and s1_vec.x*dx>=-1.0e-10:
                return Point(s2.start.x, s1.start.y+s1_m*(dx))
            return None
        if s1_m==math.inf: #segmento 1 verticale; stesse condizioni ma con s1->s2, dx->-dx, dy->-dy
            #print("Vertical ",s1_hp*(s2_m*dx-dy), s2_vec.x*dx)
            if s1_hp*(s2_m*dx-dy)<=1.0e-10 and s2_vec.x*dx<=1.0e-10:
                return Point(s1.start.x, s2.start.y-s2_m*(dx))
            return None
        det = s2_vec.x * s1_vec.y - s2_vec.y * s1_vec.x
        if det==0:
            return None
        u = (dy * s2_vec.x - dx * s2_vec.y) / det
        v = (dy * s1_vec.x - dx * s1_vec.y) / det
        #print("Not vertical ", str(u), str(v))
        if (u>=-1e-10 and v>=1e-10) or (u>=1e-10 and v>=-1e-10):
            return Point(s1.start.x+u*s1_vec.x, s1.start.y+u*s1_vec.y)

In [1450]:
s1=Segment(start=Point(0,0), pl=Point(1,0), pr=Point(0,1))
s2=Segment(start=Point(0,0), pl=Point(1,0), pr=Point(-1,0))
cross_point=s1.get_intersection(s2)
if cross_point!=None:
    print("Cross point:", cross_point.str_point())

Cross point: [0,0.0]


In [1451]:
class Beachline(object):
    
    def __init__(self, item):
        self.item = item
        self.parent = None
        self.l = None
        self.r = None
        self.c_event=None
        
    def str_node(self, directions=[]):
        cur_node=self
        for d in directions:
            cur_node=cur_node.l if d=="l" else cur_node.r
        if isinstance(cur_node.item, Segment):
            return "Edge. "+cur_node.item.str_segment()
        else:
            return "Arc. "+cur_node.item.str_point()+". Circle event: None" if cur_node.c_event==None else "Arc. "+cur_node.item.str_point()+". Event: "+cur_node.c_event.str_event()
    
    def str_tree(self, path="Path: root"):
        node_string=path+". "+self.str_node()+"\n"
        if not self.is_leaf():
            node_string+=self.l.str_tree(path+"->l")
            node_string+=self.r.str_tree(path+"->r")
        return node_string
    
    def get_leaves(self):
        cur_node=self
        leaves=[]
        while not cur_node.is_leaf():
            cur_node=cur_node.l
        while cur_node!=None:
            leaves.append(cur_node)
            cur_node=cur_node.get_rleaf_and_rparent()[0]
        return leaves
              
    def is_leaf(self):
        return self.l==None
    
    def is_lchild(self):
        return not self.is_root() and self.parent.l==self
    
    def is_rchild(self):
        return not self.is_root() and self.parent.r==self
                
    def is_root(self):
        return self.parent==None
        
    def add_rchild(self, item, event=None):
        child = Beachline(item)
        child.parent = self
        self.r = child
        
    def add_lchild(self, item, event=None):
        child = Beachline(item)
        child.parent = self
        self.l = child
    
    def get_arc_node_on_site(self, new_site):
        cur_node = self
        while isinstance(cur_node.item, Segment): #se il nuovo sito è equidistante da pl e pr è automaticamente associato al figlio destro (non so se corretto)
            #print("Intersection between",cur_node.item.pl.str_point(),"and",cur_node.item.pr.str_point(),"when q=",str(new_site.y),": ", str(arc_cross_x(cur_node.item.pl, cur_node.item.pr, new_site.y)))
            if cur_node.item.pl.y<cur_node.item.pr.y:
                sol=0
            else:
                sol=1
            cur_node = cur_node.l if new_site.x<(arc_cross_x(cur_node.item.pl, cur_node.item.pr, new_site.y)[sol]) else cur_node.r
        return cur_node
    
    def get_lleaf_and_lparent(self, leaf=None):
        cur_node=self if leaf==None else leaf #se non passo un argomento al metodo utilizza come foglia il nodo chiamante
        while cur_node.is_lchild():
            cur_node=cur_node.parent
        if cur_node.is_root():
            return [None,None]
        ledge=cur_node.parent
        cur_node=cur_node.parent.l
        while not cur_node.is_leaf():
            cur_node=cur_node.r
        return cur_node, ledge
    
    def get_rleaf_and_rparent(self, leaf=None):
        cur_node=self if leaf==None else leaf #se non passo un argomento al metodo utilizza come foglia il nodo chiamante
        while cur_node.is_rchild():
            cur_node=cur_node.parent
        if cur_node.is_root():
            return [None, None]
        redge=cur_node.parent
        cur_node=cur_node.parent.r
        while not cur_node.is_leaf():
            cur_node=cur_node.l
        return cur_node, redge
            
    #modifica la beach in modo che, dove prima c'era il sito arco intersecato, ora si ha sito arco intersecato,
    #segmento con inizio in intersezione, nuovo sito, segmento con inizio in intersezione, sito arco intersecato
    def add_arc(self, new_arc_site, crossed_arc_node):
        crossed_arc_site = crossed_arc_node.item
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: va aggiunto a new_arc_site.y un vettore di lung=new_arc_site.dist_to_par(crossed_arc_site) nel verso opposto a quello di sweep
        #se la sl va verso y crescenti la y del circle event sarà cc.y-r
        #se la sl va verso y decrescenti la y del circle event sarà cc.y+r
        cross_point = Point(new_arc_site.x, new_arc_site.y-new_arc_site.dist_to_par(crossed_arc_site))
        new_lhalfedge=Segment(pl=crossed_arc_site, pr=new_arc_site, start=cross_point)
        new_rhalfedge=Segment(pl=new_arc_site, pr=crossed_arc_site, start=cross_point)
        crossed_arc_node.item = new_lhalfedge
        crossed_arc_node.add_lchild(crossed_arc_site)
        crossed_arc_node.add_rchild(new_rhalfedge)
        crossed_arc_node.r.add_lchild(new_arc_site)
        crossed_arc_node.r.add_rchild(crossed_arc_site)
     
    def rm_arc(self, new_edge_start, arcs_nodes, edges_nodes):#in realtà potrebbe calcolarseli tutti, ma mi sembrava più efficiente passarglieli siccome già noti
        #salvataggi dei valori da ritornare prima dellaggiornamento dell'albero
        (live_larc_node, dead_arc_node, live_rarc_node)=(arcs_nodes[0], arcs_nodes[1], arcs_nodes[2])
        print(arc_nodes[0].str_node())
        #aggiornamento dell'albero
            #salvataggi dei riferimenti per nuovo assegnamento
        if dead_arc_node.is_rchild():
            (live_cut_branch, mid_edge_node, top_edge_node)=(edges_nodes[0].l, edges_nodes[0], edges_nodes[1])  
        else:
            (live_cut_branch, mid_edge_node, top_edge_node)=(edges_nodes[1].r, edges_nodes[1], edges_nodes[0])  
            #nuovo assegnamento
        top_edge_node.item = Segment(pl=live_larc_node.item, pr=live_rarc_node.item, start=new_edge_start, end=None)
        if mid_edge_node.is_rchild(): #aggiornamento puntatore padre->figlio
            mid_edge_node.parent.r=live_cut_branch #non usa metodo add_rchild perchè non aggiunge un nodo a partire da un item
        else:
            mid_edge_node.parent.l=live_cut_branch
        live_cut_branch.parent=mid_edge_node.parent #aggiornamento puntatore figlio->padre

In [1452]:
first_arc_site = Point(5, 10)
bl = Beachline(first_arc_site)

In [1453]:
print(bl.item.x, bl.item.y, bl.parent, bl.l, bl.r, bl.c_event)

5 10 None None None None


In [1454]:
second_site = Point(first_arc_site.x+2, first_arc_site.y-2) #così facendo, le halfedge sono costruite con start=[first_arc_site.x+2, first_arc_site.y]
crossed_arc_node = bl.get_arc_node_on_site(second_site)
bl.add_arc(second_site, crossed_arc_node)
print("Root. Segment with start point: [", bl.item.start.x, ",", bl.item.start.y, "]") #1st-2nd site cross point
print("Root.r. Segment with start point: [", bl.r.item.start.x, ",", bl.r.item.start.y, "]") #1st-2nd site cross point
print("Root.l. Point: [", bl.l.item.x, ",", bl.l.item.y, "]") #1st site
print("Root.r.r. Point: [", bl.r.r.item.x, ",", bl.r.r.item.y, "]") #1st site
print("Root.r.l. Point: [", bl.r.l.item.x, ",", bl.r.l.item.y, "]") #2nd site

Root. Segment with start point: [ 7 , 6.0 ]
Root.r. Segment with start point: [ 7 , 6.0 ]
Root.l. Point: [ 5 , 10 ]
Root.r.r. Point: [ 5 , 10 ]
Root.r.l. Point: [ 7 , 8 ]


In [1455]:
third_site = Point(6, 6)
crossed_arc_node = bl.get_arc_node_on_site(third_site)
bl.add_arc(third_site, crossed_arc_node)
print(bl.str_tree())

Path: root. Edge. Start: [7,6.0]; Theta: 44.98; left arc focus: [5,10]; right arc focus: [7,8]
Path: root->l. Edge. Start: [6,3.875]; Theta: 14.04; left arc focus: [5,10]; right arc focus: [6,6]
Path: root->l->l. Arc. [5,10]. Circle event: None
Path: root->l->r. Edge. Start: [6,3.875]; Theta: -165.99; left arc focus: [6,6]; right arc focus: [5,10]
Path: root->l->r->l. Arc. [6,6]. Circle event: None
Path: root->l->r->r. Arc. [5,10]. Circle event: None
Path: root->r. Edge. Start: [7,6.0]; Theta: -134.99; left arc focus: [7,8]; right arc focus: [5,10]
Path: root->r->l. Arc. [7,8]. Circle event: None
Path: root->r->r. Arc. [5,10]. Circle event: None



In [1456]:
first_leaf = bl.l.l
print(first_leaf.item.x, first_leaf.item.y)
second_leaf = bl.get_rleaf_and_rparent(first_leaf)[0]
print(second_leaf.item.x, second_leaf.item.y) #2nd site
third_leaf = bl.get_rleaf_and_rparent(second_leaf)[0]
print(third_leaf.item.x, third_leaf.item.y) #3rd site
fourth_leaf = bl.get_rleaf_and_rparent(third_leaf)[0]
print(fourth_leaf.item.x, fourth_leaf.item.y) #2nd site
fifth_leaf = bl.get_rleaf_and_rparent(fourth_leaf)[0]
print(fifth_leaf.item.x, fifth_leaf.item.y) #1st site
sixth_leaf = bl.get_rleaf_and_rparent(fifth_leaf)[0]
print(sixth_leaf) #None
print(fourth_leaf==fifth_leaf.get_lleaf_and_lparent()[0])
print(third_leaf==fourth_leaf.get_lleaf_and_lparent()[0])
print(second_leaf==third_leaf.get_lleaf_and_lparent()[0])
print(first_leaf==second_leaf.get_lleaf_and_lparent()[0])
print(first_leaf.get_lleaf_and_lparent()[0]) #None

5 10
6 6
5 10
7 8
5 10
None
True
True
True
True
None


In [1457]:
new_edge_start=Point(2,9)
(lleaf, ledge_node)=second_leaf.get_lleaf_and_lparent()
(rleaf, redge_node)=second_leaf.get_rleaf_and_rparent()
bl.rm_arc(new_edge_start, [lleaf,second_leaf,rleaf], [ledge_node, redge_node]) #second leaf è un figlio sinistro
print(bl.str_tree())

Arc. [2,0]. Circle event: None
Path: root. Edge. Start: [7,6.0]; Theta: 44.98; left arc focus: [5,10]; right arc focus: [7,8]
Path: root->l. Edge. Start: [2,9]; Theta: -90.01; left arc focus: [5,10]; right arc focus: [5,10]
Path: root->l->l. Arc. [5,10]. Circle event: None
Path: root->l->r. Arc. [5,10]. Circle event: None
Path: root->r. Edge. Start: [7,6.0]; Theta: -134.99; left arc focus: [7,8]; right arc focus: [5,10]
Path: root->r->l. Arc. [7,8]. Circle event: None
Path: root->r->r. Arc. [5,10]. Circle event: None



In [1458]:
nnew_edge_start=Point(3,4)
(lleaf, ledge_node)=third_leaf.get_lleaf_and_lparent()
(rleaf, redge_node)=third_leaf.get_rleaf_and_rparent()
bl.rm_arc(new_edge_start, [lleaf,third_leaf,rleaf], [ledge_node, redge_node]) #fifth leaf è un figlio destro
print(bl.str_tree())

Arc. [2,0]. Circle event: None
Path: root. Edge. Start: [2,9]; Theta: 44.98; left arc focus: [5,10]; right arc focus: [7,8]
Path: root->l. Arc. [5,10]. Circle event: None
Path: root->r. Edge. Start: [7,6.0]; Theta: -134.99; left arc focus: [7,8]; right arc focus: [5,10]
Path: root->r->l. Arc. [7,8]. Circle event: None
Path: root->r->r. Arc. [5,10]. Circle event: None



In [1459]:
class Event(object):
    
    def __init__(self, arc_site_or_nodes): #sites: tupla dei siti associati agli archi coinvolti nell'evento (ordinati)
        if isinstance(arc_site_or_nodes, Point): 
            self.coord=arc_site_or_nodes
            self.arcs_nodes=None
        else:
            self.arcs_nodes=[arc_site_or_nodes[0], arc_site_or_nodes[2], arc_site_or_nodes[4]]
            self.edges_nodes=[arc_site_or_nodes[1], arc_site_or_nodes[3]]
            self.vertex_coord=circumcenter(self.arcs_nodes[0].item, self.arcs_nodes[1].item, self.arcs_nodes[2].item)
            #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: se sweep verso y crescenti uso circumtop, viceversa circumbottom
            self.coord=Point(self.vertex_coord.x,self.vertex_coord.y+self.arcs_nodes[0].item.dist_to_point(self.vertex_coord))

            
    def str_event(self):
        if(self.arcs_nodes==None): #site event
            return "Site event. Site: "+self.coord.str_point()  
        else: #circle event
            return "Circle event. Coord:"+self.coord.str_point()+". Arc site:"+self.arcs_nodes[1].item.str_point()
    
    def is_same_event(self, e):
        return self.arcs_nodes==e.arcs_nodes and self.coord==e.coord
    
    def is_coord_y_greater(self, e):
        return self.coord.y > e.coord.y+1e-10
    
    #QUI DEFINISCE L'ORDINAMENTO IN Y DEGLI EVENTI: in base a come ordino gli eventi, la sl andrà verso y crescenti o decrescenti
    #inserisce i nuovi circle come primi eventi a tale y. Penso sia giusto gestirli prima dei site event, in modo da avere la beach aggiornata
    def add(self, es):  
        es.insert(go_to_i(es, self.is_coord_y_greater), self)
    
    def rm(self, es):
        i = go_to_i(es, self.is_coord_y_greater)
        while True:
            #se non esiste un altro evento alla stessa y elimina subito l'evento corrente, se no effettua il controllo
            if i==len(es)-1 or self.coord.y != es[i+1].coord.y or self.is_same_event(es[i]): 
                return es.pop(i)
            i+=1            

In [1460]:
class Voronoi(object):
    
    def __init__(self, sites):
        self.sites = sites  #siti di V, obj Point
        self.edges = []
        self.events = []
        for site in sites:
            Event(site).add(self.events)
        self.beach = Beachline(self.events[0].coord)     
    
    #rimuove dalla lista degli eventi l'evento cicle il cui arco associato all'indice della beachline argomento è 
    #quello centrale
    def rm_circle_event(self, arc_node):
        if arc_node.c_event != None:
            arc_node.c_event.rm(self.events)
            arc_node.c_event = None
    
    #NB: un circle event è originato da tre siti di V adiacenti. In esso viene eliminato l'arco centrale. Da notare
    #che l'arco centrale non è associabile a priori in modo banale ad uno dei tre punti
    #Il metodo controlla gli archi interessati da un cambiamento della beachline e verifica se ci sono dei circle
    #event loro associati. Se ci sono, aggiunge l'evento alla coda salvando i siti degli archi nell'ordine di comparsa
    #e salvando un riferimento all'evento 
    def add_circle_event(self, arc_node, sweep_y):
        larc_node = arc_node.get_lleaf_and_lparent()[0]
        rarc_node = arc_node.get_rleaf_and_rparent()[0]
        ledge_node = arc_node.get_lleaf_and_lparent()[1]
        redge_node = arc_node.get_rleaf_and_rparent()[1]
        if larc_node==None or rarc_node==None:
            print("No event: first or last arc")
            return "No event: first or last arc"
        if larc_node.item.is_same_point(rarc_node.item):
            print("No event: lefter arc site equals righter arc site")
            return "No event: lefter arc site equals righter arc site"
        cc = circumcenter(larc_node.item, arc_node.item, rarc_node.item)
        if cc.y==math.inf:
            print("No event: the three arcs sites are on the same line")
            return "No event: the three arcs sites are on the same line"
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: va preso l'estremo della circonferenza nella direzione di sweep
        #se la sl va verso y crescenti la y del circle event sarà cc.y+r
        #se la sl va verso y decrescenti la y del circle event sarà cc.y-r
        event_point=Point(cc.x, cc.y+arc_node.item.dist_to_point(cc))
        #QUI SFRUTTA L'ORDINAMENTO IN Y DEGLI EVENTI: un evento è sotto la sl se deve essere ancora incontrato durante lo sweep, e viceversa
        #se la sl va verso y crescenti un evento è sotto la sl se ha y minore di quella della sl
        #se la sl va verso y decrescenti un evento è sotto la sl se ha y maggiore di quella della sl
        if event_point.y<sweep_y-1e-10:
            print("No event: circle event y", event_point.y ,"is below sweepline y",sweep_y)
            return "No event: circle event coordinate below sweepline"
        if ledge_node.item.get_intersection(redge_node.item)==None:
            print("Edges do not cross")
            return "Edges do not cross"
        arc_node.c_event = Event([larc_node, ledge_node, arc_node, redge_node, rarc_node])
        arc_node.c_event.add(self.events)
        print("event added")
        return arc_node.c_event
        
    def manage_site_event(self):
        event=self.events.pop(0) #tolgo subito l'evento gestito; facendolo dopo potrei rimuovere un circle event aggiunto
        crossed_arc_node = self.beach.get_arc_node_on_site(event.coord)
        rarc_node=crossed_arc_node.get_rleaf_and_rparent()[0]
        larc_node=crossed_arc_node.get_lleaf_and_lparent()[0]
        self.rm_circle_event(crossed_arc_node)
        if larc_node!=None:
            self.rm_circle_event(larc_node)
        if rarc_node!=None:
            self.rm_circle_event(rarc_node)
        self.beach.add_arc(event.coord, crossed_arc_node)
        if larc_node!=None:
            self.add_circle_event(larc_node, event.coord.y)
        if rarc_node!=None:
            self.add_circle_event(rarc_node, event.coord.y)
        self.add_circle_event(crossed_arc_node.l, event.coord.y)
        self.add_circle_event(crossed_arc_node.r.r, event.coord.y)
        
    def manage_circle_event(self):
        event=self.events.pop(0) #tolgo subito l'evento gestito; facendolo dopo potrei rimuovere un circle event aggiunto
        (dead_edgeA, dead_edgeB)=(event.edges_nodes[0].item, event.edges_nodes[1].item)
        (dead_edgeA.end, dead_edgeB.end)=(event.vertex_coord, event.vertex_coord) 
        self.beach.rm_arc(event.vertex_coord, event.arcs_nodes, event.edges_nodes)
        self.edges.append(dead_edgeA)
        self.edges.append(dead_edgeB)
        self.rm_circle_event(event.arcs_nodes[0])
        self.rm_circle_event(event.arcs_nodes[2])
        self.add_circle_event(event.arcs_nodes[0], event.coord.y)
        self.add_circle_event(event.arcs_nodes[2], event.coord.y)

In [1461]:
v=Voronoi((first_leaf.item, second_leaf.item, third_leaf.item))
print("sites:", *[str([site.x, site.y]) for site in v.sites])
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #3rd site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #2nd site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #1st site
print("1st beachline item:", v.beach.item.x, v.beach.item.y) #3rd site coordinates

sites: [5, 10] [6, 6] [5, 10]
1st event site: 6 6
2nd event site: 5 10
3rd event site: 5 10
1st beachline item: 6 6


In [1462]:
v.events[1].rm(v.events) #rimozione in mezzo alla coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #3rd site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y, "\n") #1st site
Event(Point(7, 7)).add(v.events) #aggiunta in mezzo alla coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #3rd site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #1st site

1st event site: 6 6
2nd event site: 5 10 

1st event site: 6 6
2nd event site: 7 7
3rd event site: 5 10


In [1463]:
v.events[0].rm(v.events) #rimozione a inizio coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_1 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y, "\n") #1st site
Event(Point(0, -1)).add(v.events) #aggiunta a inizio coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #1st site

1st event site: 7 7
2nd event site: 5 10 

1st event site: 0 -1
2nd event site: 7 7
3rd event site: 5 10


In [1464]:
v.events[2].rm(v.events) #rimozione a fondo coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2_site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y, "\n") #new_ev_1 site
Event(Point(-6, 9.3)).add(v.events) #aggiunta a fondo coda
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #new_ev_3 site

1st event site: 0 -1
2nd event site: 7 7 

1st event site: 0 -1
2nd event site: 7 7
3rd event site: -6 9.3


In [1465]:
Event(Point(-5, 9.3)).add(v.events) #aggiunta alla stessa y 
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #new_ev_4 site
print("4th event site:", v.events[3].coord.x, v.events[2].coord.y) #new_ev_3 site

1st event site: 0 -1
2nd event site: 7 7
3rd event site: -5 9.3
4th event site: -6 9.3


In [1466]:
v.events[3].rm(v.events) #rimozione alla stessa y
print("1st event site:", v.events[0].coord.x, v.events[0].coord.y) #new_ev_2 site
print("2nd event site:", v.events[1].coord.x, v.events[1].coord.y) #new_ev_1 site
print("3rd event site:", v.events[2].coord.x, v.events[2].coord.y) #new_ev_4 site

1st event site: 0 -1
2nd event site: 7 7
3rd event site: -5 9.3


In [1467]:
### controllo di add_circle_event
v1=Voronoi([Point(0,2), Point(math.sqrt(2),math.sqrt(2)), Point(2,0)])
new_site=v1.events[1].coord
crossed_arc_node=v1.beach.get_arc_node_on_site(new_site)
v1.beach.add_arc(new_site, crossed_arc_node)
new_site=v1.events[2].coord
crossed_arc_node=v1.beach.get_arc_node_on_site(new_site)
v1.beach.add_arc(new_site, crossed_arc_node)
#new_site=v1.events[3].coord
#crossed_arc_node=v1.beach.get_arc_node_on_site(new_site)
#v1.beach.add_arc(new_site, crossed_arc_node)

In [1468]:
arc_nodes=v1.beach.get_leaves()
for arc_node in arc_nodes:
    print(arc_node.str_node([]))

Arc. [2,0]. Circle event: None
Arc. [0,2]. Circle event: None
Arc. [2,0]. Circle event: None
Arc. [1.414,1.414]. Circle event: None
Arc. [2,0]. Circle event: None


In [1469]:
print("Before checking for events")
print("\n# Arc nodes")
for arc_node in v1.beach.get_leaves():
    print(arc_node.str_node([]))
print("\n# Event queue")
print([event.str_event() for event in v1.events])

Before checking for events

# Arc nodes
Arc. [2,0]. Circle event: None
Arc. [0,2]. Circle event: None
Arc. [2,0]. Circle event: None
Arc. [1.414,1.414]. Circle event: None
Arc. [2,0]. Circle event: None

# Event queue
['Site event. Site: [2,0]', 'Site event. Site: [1.414,1.414]', 'Site event. Site: [0,2]']


In [1470]:
#non elimina gli archi intermedi anche una volta morti siccome non gestisco i circle event
#nB: coordinate dei circle event non sono il centro della circonferenza ma il suo estremo
print("\nWhile checking for circle events")
sweep_y=1 #da tunare per verificare se l'evento viene scartato quando cade nel range già scorso
for arc_node in v1.beach.get_leaves():
    print("*", v1.add_circle_event(arc_node, sweep_y))
    print(arc_node.str_node([]))
print("\n", [event.str_event() for event in v1.events])


While checking for circle events
No event: first or last arc
* No event: first or last arc
Arc. [2,0]. Circle event: None
No event: lefter arc site equals righter arc site
* No event: lefter arc site equals righter arc site
Arc. [0,2]. Circle event: None
event added
* <__main__.Event object at 0x104e048d0>
Arc. [2,0]. Event: Circle event. Coord:[0.0,2.0]. Arc site:[2,0]
No event: lefter arc site equals righter arc site
* No event: lefter arc site equals righter arc site
Arc. [1.414,1.414]. Circle event: None
No event: first or last arc
* No event: first or last arc
Arc. [2,0]. Circle event: None

 ['Site event. Site: [2,0]', 'Site event. Site: [1.414,1.414]', 'Circle event. Coord:[0.0,2.0]. Arc site:[2,0]', 'Site event. Site: [0,2]']


In [1471]:
### controllo di rm_circle_event
print("\nWhile checking for circle events")
print("\nEvent queue before removal: ")
print([event.str_event() for event in v1.events])
for arc_node in v1.beach.get_leaves():
    print("\nCircle event before removal: "+arc_node.str_node([]))
    v1.rm_circle_event(arc_node)
    print("Circle event after removal: "+arc_node.str_node([]))
print("\nEvent queue after removal: ")
print([event.str_event() for event in v1.events])


While checking for circle events

Event queue before removal: 
['Site event. Site: [2,0]', 'Site event. Site: [1.414,1.414]', 'Circle event. Coord:[0.0,2.0]. Arc site:[2,0]', 'Site event. Site: [0,2]']

Circle event before removal: Arc. [2,0]. Circle event: None
Circle event after removal: Arc. [2,0]. Circle event: None

Circle event before removal: Arc. [0,2]. Circle event: None
Circle event after removal: Arc. [0,2]. Circle event: None

Circle event before removal: Arc. [2,0]. Event: Circle event. Coord:[0.0,2.0]. Arc site:[2,0]
Circle event after removal: Arc. [2,0]. Circle event: None

Circle event before removal: Arc. [1.414,1.414]. Circle event: None
Circle event after removal: Arc. [1.414,1.414]. Circle event: None

Circle event before removal: Arc. [2,0]. Circle event: None
Circle event after removal: Arc. [2,0]. Circle event: None

Event queue after removal: 
['Site event. Site: [2,0]', 'Site event. Site: [1.414,1.414]', 'Site event. Site: [0,2]']


In [1472]:
v1=Voronoi([Point(0,0), Point(0,1), Point(0,2), Point(0,3)])

In [1473]:
#v1=Voronoi([Point(0,0), Point(1,1), Point(-1,1)])
#v1=Voronoi([Point(0,-1), Point(1,0), Point(-1,0), Point(0,1)])
#v1=Voronoi([Point(0,0), Point(1,1), Point(-1,1), Point(0,3)])
#v1=Voronoi([Point(0,-2), Point(-math.sqrt(2),-math.sqrt(2)), Point(math.sqrt(2),-math.sqrt(2)), Point(-2,0), Point(2,0),Point(-math.sqrt(2),math.sqrt(2)), Point(math.sqrt(2),math.sqrt(2)), Point(0,2)])

In [1474]:
print([event.str_event() for event in v1.events])
print("\n")

v1.events.pop(0)
for arc_node in v1.beach.get_leaves():
    print(arc_node.str_node([]))
print([event.str_event() for event in v1.events])
print("\n")

while v1.events != []:
#for i in range(3):
    if v1.events[0].arcs_nodes==None:
        print("***** Site Event")
        v1.manage_site_event()
    else:
        print("***** Circle Event")
        v1.manage_circle_event()
    print(v1.beach.str_tree())
    print([event.str_event() for event in v1.events])
    #if v1.events[-2].arcs_nodes:
        #print("last circle event: ", [node.str_node() for node in v1.events[-2].arcs_nodes])
    [print(edge.str_segment()) for edge in v1.edges]
    print("\n")

['Site event. Site: [0,0]', 'Site event. Site: [0,1]', 'Site event. Site: [0,2]', 'Site event. Site: [0,3]']


Arc. [0,0]. Circle event: None
['Site event. Site: [0,1]', 'Site event. Site: [0,2]', 'Site event. Site: [0,3]']


***** Site Event
No event: first or last arc
No event: first or last arc
Path: root. Edge. Start: [0,0.5]; Theta: 180.02; left arc focus: [0,0]; right arc focus: [0,1]
Path: root->l. Arc. [0,0]. Circle event: None
Path: root->r. Edge. Start: [0,0.5]; Theta: 0.0; left arc focus: [0,1]; right arc focus: [0,0]
Path: root->r->l. Arc. [0,1]. Circle event: None
Path: root->r->r. Arc. [0,0]. Circle event: None

['Site event. Site: [0,2]', 'Site event. Site: [0,3]']


***** Site Event
No event: first or last arc
No event: first or last arc
No event: the three arcs sites are on the same line
No event: the three arcs sites are on the same line
Path: root. Edge. Start: [0,0.5]; Theta: 180.02; left arc focus: [0,0]; right arc focus: [0,1]
Path: root->l. Arc. [0,0]. Circle eve

In [1475]:
print(v1.beach.str_tree())

Path: root. Edge. Start: [0,0.5]; Theta: 180.02; left arc focus: [0,0]; right arc focus: [0,1]
Path: root->l. Arc. [0,0]. Circle event: None
Path: root->r. Edge. Start: [0,0.5]; Theta: 0.0; left arc focus: [0,1]; right arc focus: [0,0]
Path: root->r->l. Edge. Start: [0,1.5]; Theta: 180.02; left arc focus: [0,1]; right arc focus: [0,2]
Path: root->r->l->l. Arc. [0,1]. Circle event: None
Path: root->r->l->r. Edge. Start: [0,1.5]; Theta: 0.0; left arc focus: [0,2]; right arc focus: [0,1]
Path: root->r->l->r->l. Edge. Start: [0,2.5]; Theta: 180.02; left arc focus: [0,2]; right arc focus: [0,3]
Path: root->r->l->r->l->l. Arc. [0,2]. Circle event: None
Path: root->r->l->r->l->r. Edge. Start: [0,2.5]; Theta: 0.0; left arc focus: [0,3]; right arc focus: [0,2]
Path: root->r->l->r->l->r->l. Arc. [0,3]. Circle event: None
Path: root->r->l->r->l->r->r. Arc. [0,2]. Circle event: None
Path: root->r->l->r->r. Arc. [0,1]. Circle event: None
Path: root->r->r. Arc. [0,0]. Circle event: None



In [1476]:
[print(edge.str_segment()) for edge in v1.edges]

[]

## DA GESTIRE:

### site event con nuovo sito con sulla verticale l'intersezione di due archi adiacenti 
vado a sezionare l'arco di destra (per convenzione. Avrei potuto sezionare il sinistro). Contestualmente si crea un
circle event per l'arco fittizio creato (arco più a sinistra della terna creata). Come primo evento successivo
gerstirò tale circle event aggiornando la beach

### eventi alla stessa y (sia circle che site)
penso sia meglio gestire prima i circle che i site, in modo da avere la beach aggiornata quando gestisco i site. 
Non penso più site event alla stessa y diano problemi, qualsiasi sia l'ordine con il quale sono gestiti

* Non penso più circle event alla stessa y diano problemi, anche se questo non è ovvio. In particolare andrebbe investigato il caso in cui, eliminando un arco, aggiungo un circle event (che verrà eseguito come circle event successivo, coinvolgente un arco che deve essere eliminato da un circle event alla stessa y)

### eventi alla stessa [x,y] sia circle che site
site è stupido. L'utente non dovrebbe inserirne. Nel caso si possono eliminare siti uguali dall'input fornito

* cirle event?

### più siti iniziali alla stessa y
* creo l'albero separatamente. Le foglie sono inserite controllando in quale intervallo di x cadono (e non quale è l'arco soprastante, non esistendo un arco soprastante. L'arco nel cui intervallo di x cade il nuovo sito è sostituito con un nodo a due figli. Il nodo è l'asse tra nuovo sito e sito adiacente (ci associo punto all'infinito o cosa?). Il figlio sinistro è il sito adiacente. Il figlio destro è il sito adiacente.

### chiusura algoritmo
* scandisco l'albero alla ricerca dei segmenti rimasti. Determino se sono semirette nel semipiano superiore o inferiore in base a se pl.x < pr.x o viceversa. In base a ciò decido quale intersezione con la box tenere. Tale intersezione è associata all'estremo vacante

## CAMBIAMENTI

### ottimizzazioni
* gestione puntatori in add_arc
* gestione condizione intersezione in get_intersection